# FAMA

## Pocos dias

In [12]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Dependencia opcional para Fama-French
try:
    from pandas_datareader import data as pdr
    PDR_AVAILABLE = True
except Exception as e:
    PDR_AVAILABLE = False
    _pdr_err = e

# --- Parámetros ---
tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25])
inicio = "2025-09-08"
fin = "2025-09-24"
inversion_inicial = 1_000_000 * 18.5

# --- Descargar precios ---
# yfinance puede devolver un DataFrame con niveles ("Adj Close", ticker) o con columnas simples.
# Ser robustos: descargar y preferir "Adj Close", luego "Close", y manejar Series -> DataFrame.
data = yf.download(tickers, start=inicio, end=fin, progress=False, auto_adjust=False)

if isinstance(data, pd.DataFrame) and "Adj Close" in data:
    data = data["Adj Close"].copy()
elif isinstance(data, pd.DataFrame) and "Close" in data:
    data = data["Close"].copy()
else:
    # Si yfinance devolvió una Series (un solo ticker) o ya un DataFrame de precios
    data = data.copy()

# Asegurarnos de tener un DataFrame con columnas por ticker
if isinstance(data, pd.Series):
    data = data.to_frame()

# Normalizar nombres de columnas para que coincidan con la lista 'tickers' (insensible a mayúsculas)
def normalize_col(col):
    col_str = str(col)
    for t in tickers:
        if col_str.upper() == t.upper():
            return t
        if col_str.split(".")[0].upper() == t.split(".")[0].upper():
            return t
    return col

data.columns = [normalize_col(c) for c in data.columns]

# --- Rendimientos diarios ---
rend_diarios = data.pct_change().dropna()

# --- Rendimiento acumulado por activo ---
rend_acum = (1 + rend_diarios).prod() - 1

# --- Rendimiento portafolio ---
rend_port_diario = rend_diarios @ pesos
rend_port_total = (1 + rend_port_diario).prod() - 1

# --- Riesgo (desviación estándar) ---
riesgo_port = rend_port_diario.std() * np.sqrt(len(rend_port_diario))  # aprox anualizado corto

# --- Valor final ---
valor_final = inversion_inicial * (1 + rend_port_total)

print("\n--- Resultados ---")
print("Rendimiento acumulado por activo (%):")
print(rend_acum * 100)
print("\nRendimiento total portafolio: {:.2f}%".format(rend_port_total * 100))
print("Riesgo (desviación estándar aprox.): {:.2f}%".format(riesgo_port * 100))
print("Valor final del portafolio: ${:,.2f}".format(valor_final))

# --- Fama-French 3 Factores (diario) ---
# Descargamos factores de Fama-French desde pandas_datareader ('famafrench')
if not PDR_AVAILABLE:
    # Intento de instalación automática (opcional)
    import sys, subprocess
    try:
        print("\nInstalando pandas_datareader ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas_datareader", "-q"])
        from pandas_datareader import data as pdr
        PDR_AVAILABLE = True
        print("pandas_datareader instalado correctamente.")
    except Exception as e:
        print("No se pudo importar/instalar pandas_datareader. Instálalo manualmente con: pip install pandas_datareader")
        raise e

# Descargar factores diarios
ff = pdr.DataReader("F-F_Research_Data_Factors_Daily", "famafrench")
ff_df = ff[0].copy()

# Asegurar índice datetime y usar solo columnas necesarias
ff_df.index = pd.to_datetime(ff_df.index)
ff_df = ff_df[["Mkt-RF", "SMB", "HML", "RF"]].astype(float) / 100.0  # pasar de % a decimales

# Alinear con retornos del portafolio (usar el factor más reciente disponible antes de cada fecha del portafolio)
# Esto evita que la intersección de fechas quede vacía cuando los factores no llegan hasta las mismas fechas.
rp = rend_port_diario.copy().rename("Rp")

# Preparar dataframes para merge_asof (ambos ordenados por fecha)
rp_df = rp.reset_index().rename(columns={"index": "Date"})
ff_reset = ff_df.reset_index().rename(columns={"index": "Date"}).sort_values("Date")
rp_df = rp_df.sort_values("Date")

# Para cada fecha de rp, tomar los factores de la fecha más reciente <= esa fecha
df_ff3 = pd.merge_asof(rp_df, ff_reset, on="Date", direction="backward")

# Si aún hay NaNs (por ejemplo si no existen factores previos a alguna fecha), intentar intersección clásica
if df_ff3[["Mkt-RF", "SMB", "HML", "RF"]].isnull().any(axis=None):
    df_ff3 = pd.concat([rp, ff_df], axis=1, join="inner").dropna()
    # volver a indexar por fecha
    if not df_ff3.empty:
        df_ff3.index = pd.to_datetime(df_ff3.index)

# Si seguimos sin datos alineados, informar y saltar la regresión
if df_ff3.empty:
    print("\nNo hay fechas solapadas entre los retornos del portafolio y los factores Fama-French. La regresión no puede ejecutarse.")
else:
    # Asegurar índice datetime si usamos merge_asof
    if "Date" in df_ff3.columns:
        df_ff3.set_index("Date", inplace=True)

    # Exceso de retorno del portafolio
    df_ff3["Rp_excess"] = df_ff3["Rp"] - df_ff3["RF"]

    # Regresión OLS: Rp - RF = alpha + beta_m*(Mkt-RF) + s*SMB + h*HML + error
    X = df_ff3[["Mkt-RF", "SMB", "HML"]]
    # Forzar adición de constante de forma explícita (esto crea la columna 'const')
    X = sm.add_constant(X, has_constant="add")
    y = df_ff3["Rp_excess"]

    # Comprobar que X e y no están vacíos
    if X.shape[0] == 0 or y.shape[0] == 0:
        print("\nDatos insuficientes para estimar el modelo (X o y vacíos).")
    else:
        modelo_ff3 = sm.OLS(y, X).fit()

        print("\n--- Modelo Fama-French 3 Factores (con datos diarios) ---")
        print(modelo_ff3.summary())

        # Obtener alfa de forma segura (evitar KeyError si el nombre del intercepto difiere o no existe)
        if "const" in modelo_ff3.params.index:
            alpha_diaria = modelo_ff3.params["const"]
        elif "Intercept" in modelo_ff3.params.index:
            alpha_diaria = modelo_ff3.params["Intercept"]
        else:
            # Si no hay intercepto en el modelo, aproximamos alfa como la media de los residuales
            alpha_diaria = float(modelo_ff3.resid.mean())

        alpha_anual = (1 + alpha_diaria) ** 252 - 1
        print("\nAlfa diaria: {:.4f}%".format(alpha_diaria * 100))
        print("Alfa anualizada (aprox): {:.2f}%".format(alpha_anual * 100))

        # Extraer betas de forma robusta (si faltara alguna, quedará como NaN)
        betas = modelo_ff3.params.reindex(["Mkt-RF", "SMB", "HML"])
        print("\nBetas estimadas (Mkt-RF, SMB, HML):")
        print(betas)


--- Resultados ---
Rendimiento acumulado por activo (%):
GCARSOA1.MX      4.408903
GFINBURO.MX      2.788692
HOTEL.MX         2.339179
PINFRA.MX        4.082304
TLEVISACPO.MX   -3.710755
dtype: float64

Rendimiento total portafolio: 1.67%
Riesgo (desviación estándar aprox.): 2.83%
Valor final del portafolio: $18,809,871.55


C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_29012\3014552003.py:90: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader("F-F_Research_Data_Factors_Daily", "famafrench")



--- Modelo Fama-French 3 Factores (con datos diarios) ---
                            OLS Regression Results                            
Dep. Variable:              Rp_excess   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 29 Sep 2025   Prob (F-statistic):                nan
Time:                        19:28:29   Log-Likelihood:                 33.487
No. Observations:                  10   AIC:                            -64.97
Df Residuals:                       9   BIC:                            -64.67
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

c:\Users\Juan Marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [13]:
# Resumen de rendimientos y riesgo por acción y del portafolio
# Usa variables ya definidas en el notebook: rend_diarios, rend_acum, pesos, inversion_inicial, rend_port_diario, rend_port_total

ANNUAL_FACTOR = 252

# Por acción
mean_daily = rend_diarios.mean()
var_daily = rend_diarios.var(ddof=1)
std_daily = rend_diarios.std(ddof=1)

var_annual = var_daily * ANNUAL_FACTOR
std_annual = std_daily * np.sqrt(ANNUAL_FACTOR)

valor_inicial_act = pd.Series(pesos * inversion_inicial, index=rend_diarios.columns)
valor_final_act = valor_inicial_act * (1 + rend_acum)

df_activos = pd.DataFrame({
    "Rend_acum": rend_acum,
    "Mean_daily": mean_daily,
    "Var_daily": var_daily,
    "Std_daily": std_daily,
    "Var_annual": var_annual,
    "Std_annual": std_annual,
    "Valor_inicial": valor_inicial_act,
    "Valor_final": valor_final_act
})

# Portafolio
mean_daily_port = rend_port_diario.mean()
var_daily_port = rend_port_diario.var(ddof=1)
std_daily_port = rend_port_diario.std(ddof=1)

cov_daily = rend_diarios.cov(ddof=1)
cov_annual = cov_daily * ANNUAL_FACTOR

# Varianza y std del portafolio por pesos (anualizadas desde la covarianza)
var_port_annual_from_cov = float(pesos @ cov_annual.values @ pesos)
std_port_annual_from_cov = np.sqrt(var_port_annual_from_cov)

# También cálculo directo sobre la serie del portafolio (anualizado)
var_port_annual_from_series = var_daily_port * ANNUAL_FACTOR
std_port_annual_from_series = std_daily_port * np.sqrt(ANNUAL_FACTOR)

valor_final_port = inversion_inicial * (1 + float(rend_port_total))

df_portfolio = pd.Series({
    "Rend_acum": float(rend_port_total),
    "Mean_daily": float(mean_daily_port),
    "Var_daily": float(var_daily_port),
    "Std_daily": float(std_daily_port),
    "Var_annual_from_series": float(var_port_annual_from_series),
    "Std_annual_from_series": float(std_port_annual_from_series),
    "Var_annual_from_cov": var_port_annual_from_cov,
    "Std_annual_from_cov": std_port_annual_from_cov,
    "Valor_inicial": inversion_inicial,
    "Valor_final": valor_final_port
})

# Mostrar resultados (porcentajes donde aplica)
pd.options.display.float_format = "{:,.6f}".format
print("\n--- Rendimientos y riesgo POR ACCIÓN ---")
display(df_activos.assign(
    Rend_acum_pct = df_activos["Rend_acum"] * 100,
    Mean_daily_pct = df_activos["Mean_daily"] * 100,
    Std_daily_pct = df_activos["Std_daily"] * 100,
    Std_annual_pct = df_activos["Std_annual"] * 100
)[[
    "Rend_acum_pct","Mean_daily_pct","Var_daily","Std_daily_pct","Var_annual","Std_annual_pct","Valor_inicial","Valor_final"
]])

print("\n--- Rendimiento y riesgo DEL PORTAFOLIO ---")
display(df_portfolio.rename(index={
    "Rend_acum":"Rend_acum",
    "Mean_daily":"Mean_daily",
    "Var_daily":"Var_daily",
    "Std_daily":"Std_daily"
}).to_frame(name="Portafolio").assign(
    Rend_acum_pct = df_portfolio["Rend_acum"] * 100,
    Mean_daily_pct = df_portfolio["Mean_daily"] * 100,
    Std_daily_pct = df_portfolio["Std_daily"] * 100,
    Std_annual_from_series_pct = df_portfolio["Std_annual_from_series"] * 100,
    Std_annual_from_cov_pct = df_portfolio["Std_annual_from_cov"] * 100
)[[
    "Rend_acum_pct","Mean_daily_pct","Var_daily","Std_daily_pct",
    "Var_annual_from_series","Std_annual_from_series_pct",
    "Var_annual_from_cov","Std_annual_from_cov_pct",
    "Valor_inicial","Valor_final"
]])


--- Rendimientos y riesgo POR ACCIÓN ---


,Rend_acum_pct,Mean_daily_pct,Var_daily,Std_daily_pct,Var_annual,Std_annual_pct,Valor_inicial,Valor_final
GCARSOA1.MX,4.408903,0.456071,0.000530,2.301593,0.133493,36.536656,"1,850,000.000000","1,931,564.708818"
GFINBURO.MX,2.788692,0.291181,0.000348,1.864376,0.087593,29.596056,"3,700,000.000000","3,803,181.608963"
HOTEL.MX,2.339179,0.232558,0.000024,0.487665,0.005993,7.741434,"2,775,000.000000","2,839,912.217349"
PINFRA.MX,4.082304,0.407612,0.000149,1.219086,0.037451,19.352386,"5,550,000.000000","5,776,567.859414"
TLEVISACPO.MX,-3.710755,-0.364609,0.000286,1.691872,0.072133,26.857640,"4,625,000.000000","4,453,377.588146"



--- Rendimiento y riesgo DEL PORTAFOLIO ---


KeyError: "['Var_daily', 'Var_annual_from_series', 'Var_annual_from_cov', 'Valor_inicial', 'Valor_final'] not in index"

## Un año

In [11]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Dependencia opcional para Fama-French
try:
    from pandas_datareader import data as pdr
    PDR_AVAILABLE = True
except Exception as e:
    PDR_AVAILABLE = False
    _pdr_err = e

# --- Parámetros ---
tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25])
inicio = "2025-09-08"
fin = "2025-09-24"
inversion_inicial = 1_000_000 * 18.5

# --- Descargar precios ---
# yfinance puede devolver un DataFrame con niveles ("Adj Close", ticker) o con columnas simples.
# Ser robustos: descargar y preferir "Adj Close", luego "Close", y manejar Series -> DataFrame.
data = yf.download(tickers, start=inicio, end=fin, progress=False, auto_adjust=False)

if isinstance(data, pd.DataFrame) and "Adj Close" in data:
    data = data["Adj Close"].copy()
elif isinstance(data, pd.DataFrame) and "Close" in data:
    data = data["Close"].copy()
else:
    # Si yfinance devolvió una Series (un solo ticker) o ya un DataFrame de precios
    data = data.copy()

# Asegurarnos de tener un DataFrame con columnas por ticker
if isinstance(data, pd.Series):
    data = data.to_frame()

# Normalizar nombres de columnas para que coincidan con la lista 'tickers' (insensible a mayúsculas)
def normalize_col(col):
    col_str = str(col)
    for t in tickers:
        if col_str.upper() == t.upper():
            return t
        if col_str.split(".")[0].upper() == t.split(".")[0].upper():
            return t
    return col

data.columns = [normalize_col(c) for c in data.columns]

# --- Rendimientos diarios ---
rend_diarios = data.pct_change().dropna()

# --- Rendimiento acumulado por activo ---
rend_acum = (1 + rend_diarios).prod() - 1

# --- Rendimiento portafolio (ventana de reporte) ---
rend_port_diario = rend_diarios @ pesos
rend_port_total = (1 + rend_port_diario).prod() - 1

# --- Riesgo (desviación estándar) ---
riesgo_port = rend_port_diario.std() * np.sqrt(len(rend_port_diario))  # aprox anualizado corto

# --- Valor final ---
valor_final = inversion_inicial * (1 + rend_port_total)

print("\n--- Resultados (ventana de reporte) ---")
print("Rendimiento acumulado por activo (%):")
print((rend_acum * 100).round(3))
print("\nRendimiento total portafolio: {:.2f}%".format(rend_port_total * 100))
print("Riesgo (desviación estándar aprox.): {:.2f}%".format(riesgo_port * 100))
print("Valor final del portafolio: ${:,.2f}".format(valor_final))

# --- Fama-French 3 Factores (diario) ---
# Descargamos factores de Fama-French desde pandas_datareader ('famafrench')
if not PDR_AVAILABLE:
    # Intento de instalación automática (opcional)
    import sys, subprocess
    try:
        print("\nInstalando pandas_datareader ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas_datareader", "-q"])
        from pandas_datareader import data as pdr
        PDR_AVAILABLE = True
        print("pandas_datareader instalado correctamente.")
    except Exception as e:
        print("No se pudo importar/instalar pandas_datareader. Instálalo manualmente con: pip install pandas_datareader")
        raise e

# Descargar factores diarios (EE. UU.) y preparar dataframe
ff = pdr.DataReader("F-F_Research_Data_Factors_Daily", "famafrench")
ff_df = ff[0].copy()
ff_df.index = pd.to_datetime(ff_df.index)
ff_df = ff_df[["Mkt-RF", "SMB", "HML", "RF"]].astype(float) / 100.0  # pasar de % a decimales

# Funciones auxiliares
from pandas.tseries.offsets import BDay

def build_ff3_dataset(rp_series: pd.Series, ff_factors: pd.DataFrame) -> pd.DataFrame:
    rp_series = rp_series.copy().rename("Rp")
    df = pd.concat([rp_series, ff_factors], axis=1, join="inner").dropna()
    if df.empty:
        return df
    df["Rp_excess"] = df["Rp"] - df["RF"]
    return df

def clean_and_select_features(df: pd.DataFrame, base_features=None, min_std=1e-10):
    if base_features is None:
        base_features = ["Mkt-RF", "SMB", "HML"]
    # descartar columnas con varianza casi cero
    kept = [c for c in base_features if c in df.columns and df[c].std(skipna=True) > min_std]
    return kept

# Construir dataset en ventana de reporte
features = ["Mkt-RF", "SMB", "HML"]
rp_report = rend_port_diario
ff3_report = build_ff3_dataset(rp_report, ff_df)

# Criterios mínimos para estimación
MIN_OBS = 60  # p.ej., al menos ~3 meses de datos diarios
used_window = "reporte"
ff3_used = ff3_report

# Si no alcanza la muestra o hay singularidad, ampliar ventana de estimación
need_extend = len(ff3_report) < MIN_OBS

if not need_extend and not ff3_report.empty:
    # Checar rango/singularidad preliminar
    feats0 = clean_and_select_features(ff3_report, features)
    X0 = sm.add_constant(ff3_report[feats0]) if feats0 else None
    if X0 is None or np.linalg.matrix_rank(X0.values) < X0.shape[1]:
        need_extend = True

if need_extend:
    inicio_dt = pd.to_datetime(inicio)
    # retroceder 252 días hábiles (~1 año)
    inicio_est = (inicio_dt - BDay(252)).date().isoformat()
    print(f"\n[Info] Ampliando ventana de estimación: {inicio_est} -> {fin} para estimar betas (min {MIN_OBS} obs)")
    data_est = yf.download(tickers, start=inicio_est, end=fin, progress=False, auto_adjust=False)
    if isinstance(data_est, pd.DataFrame) and "Adj Close" in data_est:
        data_est = data_est["Adj Close"].copy()
    elif isinstance(data_est, pd.DataFrame) and "Close" in data_est:
        data_est = data_est["Close"].copy()
    if isinstance(data_est, pd.Series):
        data_est = data_est.to_frame()
    data_est.columns = [normalize_col(c) for c in data_est.columns]
    rend_est = data_est.pct_change().dropna()
    rp_est = (rend_est @ pesos).rename("Rp")
    ff3_est = build_ff3_dataset(rp_est, ff_df)
    ff3_used = ff3_est
    used_window = "ampliada"

# Validaciones y limpieza de variables explicativas
if ff3_used.empty:
    raise ValueError("No se pudo construir la muestra para la regresión FF3 (sin datos tras alinear fechas).")

sel_feats = clean_and_select_features(ff3_used, features)
if not sel_feats:
    raise ValueError("Todas las variables FF tienen varianza ~0 en la ventana. Imposible estimar.")

# Intento iterativo para evitar singularidad: si la matriz no tiene rango completo, ir quitando factores
feats_try = sel_feats.copy()
while True:
    X = sm.add_constant(ff3_used[feats_try])
    rank = np.linalg.matrix_rank(X.values)
    if rank == X.shape[1]:
        break
    if len(feats_try) == 1:
        # No se puede reducir más (CAPM con Mkt-RF suele ser estable)
        break
    # Heurística: quitar la columna más correlacionada con las demás
    corr = ff3_used[feats_try].corr().abs()
    np.fill_diagonal(corr.values, 0)
    # columna con mayor suma de correlaciones absolutas
    col_to_drop = corr.sum().idxmax()
    feats_try.remove(col_to_drop)

# Ajuste del modelo
X = sm.add_constant(ff3_used[feats_try])
y = ff3_used["Rp_excess"]
modelo_ff3 = sm.OLS(y, X).fit()

print("\n--- Modelo Fama-French 3 Factores (datos diarios) ---")
print(f"Ventana usada: {used_window} | Observaciones: {len(ff3_used)} | Factores: {feats_try}")
print(modelo_ff3.summary())

alpha_diaria = float(modelo_ff3.params.get("const", np.nan))
alpha_anual = (1 + alpha_diaria) ** 252 - 1 if np.isfinite(alpha_diaria) else np.nan
print("\nAlfa diaria: {}".format("{:.4f}%".format(alpha_diaria * 100) if np.isfinite(alpha_diaria) else "NA"))
print("Alfa anualizada (aprox): {}".format("{:.2f}%".format(alpha_anual * 100) if np.isfinite(alpha_anual) else "NA"))

betas = modelo_ff3.params.drop(labels=["const"], errors="ignore")
print("\nBetas estimadas:")
print(betas)


--- Resultados (ventana de reporte) ---
Rendimiento acumulado por activo (%):
GCARSOA1.MX      4.409
GFINBURO.MX      2.789
HOTEL.MX         2.339
PINFRA.MX        4.082
TLEVISACPO.MX   -3.711
dtype: float64

Rendimiento total portafolio: 1.67%
Riesgo (desviación estándar aprox.): 2.83%
Valor final del portafolio: $18,809,871.55


C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_29012\2886929774.py:90: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader("F-F_Research_Data_Factors_Daily", "famafrench")



[Info] Ampliando ventana de estimación: 2024-09-19 -> 2025-09-24 para estimar betas (min 60 obs)

--- Modelo Fama-French 3 Factores (datos diarios) ---
Ventana usada: ampliada | Observaciones: 208 | Factores: ['Mkt-RF', 'SMB', 'HML']
                            OLS Regression Results                            
Dep. Variable:              Rp_excess   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     12.06
Date:                Mon, 29 Sep 2025   Prob (F-statistic):           2.67e-07
Time:                        19:24:36   Log-Likelihood:                 622.87
No. Observations:                 208   AIC:                            -1238.
Df Residuals:                     204   BIC:                            -1224.
Df Model:                           3                                         
Covariance Type:            nonrobust                 

In [3]:
data

,GCARSOA1.MX,GFINBURO.MX,HOTEL.MX,PINFRA.MX,TLEVISACPO.MX
Date,,,,,
2025-09-08,128.149994,50.919998,3.42,243.000000,10.51
2025-09-09,126.379997,51.049999,3.44,243.889999,10.37
2025-09-10,123.029999,50.500000,3.48,245.759995,10.21
2025-09-11,127.739998,51.740002,3.50,250.399994,10.15
2025-09-12,127.019997,52.299999,3.50,251.559998,10.45
2025-09-15,130.699997,53.220001,3.50,250.160004,10.37
2025-09-17,131.809998,52.090000,3.48,247.979996,10.25
2025-09-18,132.000000,52.529999,3.50,249.940002,9.98
2025-09-19,136.279999,50.799999,3.50,245.080002,9.89


In [5]:
X

,const,Mkt-RF,SMB,HML
Date,,,,
2025-09-09,1.0,-0.0034,-0.0035,-0.0077
2025-09-10,1.0,-0.0034,-0.0035,-0.0077
2025-09-11,1.0,-0.0034,-0.0035,-0.0077
2025-09-12,1.0,-0.0034,-0.0035,-0.0077
2025-09-15,1.0,-0.0034,-0.0035,-0.0077
2025-09-17,1.0,-0.0034,-0.0035,-0.0077
2025-09-18,1.0,-0.0034,-0.0035,-0.0077
2025-09-19,1.0,-0.0034,-0.0035,-0.0077
2025-09-22,1.0,-0.0034,-0.0035,-0.0077


In [6]:
y

Date
2025-09-09   -0.002425
2025-09-10   -0.004818
2025-09-11    0.013596
2025-09-12    0.010180
2025-09-15    0.002632
2025-09-17   -0.009962
2025-09-18   -0.001719
2025-09-19   -0.011632
2025-09-22    0.009707
2025-09-23    0.009426
Name: Rp_excess, dtype: float64

In [4]:
rp

Date
2025-09-09   -0.002225
2025-09-10   -0.004618
2025-09-11    0.013796
2025-09-12    0.010380
2025-09-15    0.002832
2025-09-17   -0.009762
2025-09-18   -0.001519
2025-09-19   -0.011432
2025-09-22    0.009907
2025-09-23    0.009626
Name: Rp, dtype: float64

In [8]:
betas

Mkt-RF   -0.000005
SMB      -0.000005
HML      -0.000012
dtype: float64

In [10]:
df_ff3

,Rp,Mkt-RF,SMB,HML,RF,Rp_excess
Date,,,,,,
2025-09-09,-0.002225,-0.0034,-0.0035,-0.0077,0.0002,-0.002425
2025-09-10,-0.004618,-0.0034,-0.0035,-0.0077,0.0002,-0.004818
2025-09-11,0.013796,-0.0034,-0.0035,-0.0077,0.0002,0.013596
2025-09-12,0.010380,-0.0034,-0.0035,-0.0077,0.0002,0.010180
2025-09-15,0.002832,-0.0034,-0.0035,-0.0077,0.0002,0.002632
2025-09-17,-0.009762,-0.0034,-0.0035,-0.0077,0.0002,-0.009962
2025-09-18,-0.001519,-0.0034,-0.0035,-0.0077,0.0002,-0.001719
2025-09-19,-0.011432,-0.0034,-0.0035,-0.0077,0.0002,-0.011632
2025-09-22,0.009907,-0.0034,-0.0035,-0.0077,0.0002,0.009707
